# The barotropic vorticity equation

The barotropic vorticity equation (BVE) is given by

$$\frac{\partial \zeta}{\partial t}+\mathbf u \nabla\zeta=0$$

It can be interpreted as describing the 2D advection of vorticity $\zeta$. The advection speed is the geostrophic wind, which is given by

$$\mathbf u=\mathbf k \times \nabla \psi,$$

where $\mathbf k$ is the vertical unit vector, and $\psi$ is the streamfunction, which is related to the vorticity as

$$\zeta=\nabla^2\psi.$$

This means that the advection speed depends on the vorticity, making this a *nonlinear* problem.


## Libraries and auxiliary functions

First, numpy and matplotlib are loaded, and a function is defined to create animations.

In [ ]:
## Load numpy and matplotlib libraries
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.ioff()
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 100  # reduce to generate figures faster (but smaller)

# function for animating results
def create_animation(psi,frames=None,levels=None):
    
    # create figure with axes
    fig,ax=plt.subplots()

    # determine frames to plot
    if frames is None:
        # number of timesteps
        nt=psi.shape[0]-1
        frames=np.arange(0,nt+1)
    else:
        nt=frames[-1]
    
    # determine levels
    if levels is None:
        # generate nice levels between min and max of psi
        nLevels=20
        zmin=1.2*np.nanmin(psi[0])-0.2*np.nanmax(psi[0])
        zmax=1.2*np.nanmax(psi[0])-0.2*np.nanmin(psi[0])
        s=(zmax-zmin)/nLevels  # first guess
        ss=0.7*s/10**np.floor(np.log10(s))  # between 1 and 10
        if ss<2:
            ss=2
        elif ss<5:
            ss=5
        else:
            ss=10
        s=ss*10**np.floor(np.log10(s))
        levels=s*np.arange(np.floor(zmin/s),np.ceil(zmax/s)+1)

        
    h = ax.axis([x[0],x[-1],y[0],y[-1]])
    
    # define plot function for a single time step
    def plotResults(it):
        ax.clear()
        ax.set_title('%3i (%f)'%(it,t[it]))
        cs=ax.contour(xg,yg,psi[it],levels)
        ax.clabel(cs, cs.levels, inline=True);
    
    # animate over timesteps
    ani = animation.FuncAnimation(fig, plotResults, frames=frames,cache_frame_data=False,blit=True)
    display(ani)

## Solving the BVE

The code below solves the BVE with a leapfrog time scheme and centered 2nd order finite differences.
The forecasted field is the streamfunction; each timestep the following calculations are made:

1. Compute vorticity from the streamfunction:

$$\zeta=\nabla^2\psi$$

2. Compute the time derivative of the vorticity:

$$\frac{\partial \zeta}{\partial t}=-J(\psi,\zeta)$$

where $J(p,q)$ is the Jacobian operator:

$$J(p,q)=\frac{\partial p}{\partial x}\frac{\partial q}{\partial y}-\frac{\partial p}{\partial y}\frac{\partial q}{\partial x}$$

3. Compute the time derivative of the streamfunction:

$$\frac{\partial \psi}{\partial t}=\nabla^{-2}\frac{\partial \zeta}{\partial t}$$

where the inverse Laplacian $\nabla^{-2}$ is computed using a spectral method. Suppose we write

$$\psi=\sum_{k_x,k_y}\psi_{k_x,k_y}e^{i (k_x x+k_y y)}$$

then

$$\begin{aligned}
    \nabla^2\psi&=\frac{\partial^2\psi}{\partial x^2}+\frac{\partial^2\psi}{\partial y^2}\\
    &=-\sum_{k_x,k_y}(k_x^2+k_y^2)\psi_{k_x,k_y}e^{i (k_x x+k_y y)}
   \end{aligned}$$

i.e. each spectral coefficient is multiplied with $-(k_x^2+k_y^2)$.

Therefore, the inverse Laplacian can be computed in spectral space by *dividing* each spectral coefficient by $-(k_x^2+k_y^2)$:

$$\nabla^{-2}\zeta=-\sum_{k_x,k_y}\frac{1}{(k_x^2+k_y^2)}\zeta_{k_x,k_y}e^{i (k_x x+k_y y)}$$



In [ ]:
# parameters
nt=500
dt=1.0
nx=72
dx=1.0
ny=64
dy=1.0

# coordinates
x=np.arange(nx)*dx
y=np.arange(ny)*dy
t=np.arange(nt+1)*dt

# grid
xg,yg=np.meshgrid(x,y)

# allocate solution (stream function)
psi=np.zeros((nt+1,ny,nx))

# grid indices
jx=np.arange(nx)
jxL=(jx-1)%nx
jxR=(jx+1)%nx
jy=np.arange(ny)
jyL=(jy-1)%ny
jyR=(jy+1)%ny

# wavenumbers
kx=(np.arange(nx)+nx//2)%nx-nx//2; kx=2*np.pi/(nx*dx)*kx
ky=(np.arange(ny)+ny//2)%ny-ny//2; ky=2*np.pi/(ny*dy)*ky
kxg,kyg=np.meshgrid(kx,ky)

# initial states

# gaussian bell + harmonic function
#psi[0]=5*(np.exp(-40*( (xg/(nx*dx)-0.5)**2+(yg/(ny*dy)-0.5)**2 )) + np.sin(2*np.pi*xg/(nx*dx)))

# gaussian bell
#psi[0]=np.exp(-40*( (xg/(nx*dx)-0.5)**2+(yg/(ny*dy)-0.5)**2 ))

# single harmonic function
#psi[0]=np.cos(2*np.pi*xg/(nx*dx))*np.cos(4*np.pi*yg/(ny*dy))

# random field with gaussian spectrum
psi[0]=nx*ny*np.real(np.fft.ifft2(
    np.exp(-30*(kxg**2+kyg**2))
    *np.exp(1j*2*np.pi*np.random.rand(ny,nx))
))

# Jacobian function of psi and zeta
def jacobian(psi,zeta):
    # calculate d(psi)/dx*d(zeta)/dy-d(psi)/dy*d(zeta)/dx
    J=(
        (  psi[:,jxR]-psi[:,jxL] )*( zeta[jyR,:]-zeta[jyL,:] )
        -( psi[jyR,:]-psi[jyL,:] )*( zeta[:,jxR]-zeta[:,jxL] )
    )/(4*dx*dy)
    return(J)

# Laplacian operator
def laplacian(psi):
    zeta=(psi[:,jxL]-2*psi[:,jx]+psi[:,jxR])/dx**2+(psi[jyL,:]-2*psi[jy,:]+psi[jyR,:])/dy**2
    return(zeta)

# inverse Laplacian operator (with FFT)
def invlaplacian(zeta):
    ZS=np.fft.fft2(zeta)  # 2d fft to go from gridpoint values to spectral coefficients
    L=(np.sin(kxg*dx/2)/(dx/2))**2+(np.sin(kyg*dy/2)/(dy/2))**2  # multiplier to calculate Laplacian in spectral space
    # bonus question for students: why don't we use kxg**2+kyg**2 in the line above?
    ZS=-ZS/np.maximum(L,1.e-16)   # the np.maximum is to avoid division by zero
    ZS[(kxg==0) & (kyg==0)]=0  # the mean value (kx=ky=0) remains zero
    psi=np.real(np.fft.ifft2(ZS))  # 2d inverse fft to go back to gridpoint values
    return(psi)

# auxiliary function to calculate maximum wind in the domain
def maxwind(psi):
    U=np.sqrt( ((psi[jyR,:]-psi[jyL,:])/(2*dy))**2+((psi[:,jxR]-psi[:,jxL])/(2*dx))**2 )
    return(np.max(U))
    
# first timestep
jt=0
print('timestep ',jt,': max windspeed = %6.3f'%maxwind(psi[jt]))
# compute vorticity
zeta      = laplacian(psi[jt])
# compute vorticity time derivative
dzetadt   = -jacobian(psi[jt],zeta)
# compute stream function time derivative
dpsidt    = invlaplacian(dzetadt)
# forward time scheme
psi[jt+1] = psi[jt]+dt*dpsidt

# later timesteps
for jt in range(1,nt):
    # compute vorticity
    zeta    = laplacian(psi[jt])
    # compute vorticity time derivative
    dzetadt = -jacobian(psi[jt],zeta)
    # compute stream function time derivative
    dpsidt=invlaplacian(dzetadt)
    # leap frog time scheme
    psi[jt+1]=psi[jt-1]+2*dt*dpsidt
    
    # track max wind speed
    if jt%50==0:
        print('timestep ',jt,': max windspeed = %6.3f'%maxwind(psi[jt+1]))
    
# print final wind speed
jt=nt
print('timestep ',jt,': max windspeed = %6.3f'%maxwind(psi[jt]))

In [ ]:
# create animation with about 50 frames
create_animation(psi,frames=np.arange(0,nt+1,nt//50))